In [1]:
import numpy as np
import pandas as pd
import pprint as pp
import itertools
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("../data/moons_data.csv")

In [3]:
data.head()

,x1,x2,y
0,0.574815,0.039850,0
1,1.245934,-0.133064,0
2,0.895264,0.181897,0
3,0.981820,0.035300,0
4,1.192523,0.115514,0


In [719]:
class decisionTree:
    def __init__(self, min_leaf = 5):
        self.min_leaf = min_leaf
        return None
    
    def fit(self, X, y):
        self.dtree = Node(X, y, np.array(np.arange(len(y))), self.min_leaf)
        return self

    def predict(self, X):
        return self.dtree.predict(X)

In [748]:
class Node:

    def __init__(self, x, y, idxs, min_leaf=5, method = "classification"):
        self.x = x 
        self.y = y
        self.idxs = idxs 
        self.min_leaf = min_leaf
        self.row_count = len(idxs)
        self.col_count = x.shape[1]
        if method == "classification":
            counts = np.bincount(y[idxs])
            self.val = np.argmax(counts)
        if method == "regression":
             self.val = np.mean(y[idxs])
        self.score = float('inf')
        self.find_varsplit()
        
    def find_varsplit(self):
        for c in range(self.col_count): self.find_better_split(c)
        if self.is_leaf: return None
        x = self.split_col
        lhs = np.nonzero(x <= self.split)[0]
        rhs = np.nonzero(x > self.split)[0]
        self.lhs = Node(self.x, self.y, self.idxs[lhs], self.min_leaf)
        self.rhs = Node(self.x, self.y, self.idxs[rhs], self.min_leaf)
        
    def find_better_split(self, var_idx):
      
        x = self.x[self.idxs, var_idx]
        
        quantiles = np.percentile(x, [.75, .50, .25])
        
        for r in quantiles:
            lhs = x <= r
            rhs = x > r
            if len(rhs) < self.min_leaf or len(lhs) < self.min_leaf: continue

            curr_score = self.find_score(lhs, rhs)
            if curr_score < self.score: 
                self.var_idx = var_idx
                self.score = curr_score
                self.split = r
                
    def entropy(self, y):
        n_class = np.unique(y)
        ent = []
        for k in n_class:
            p = len(np.where(y == k)[0])/len(y)
            ent.append(-p*np.log(p))
        return np.sum(ent) 
    
    def find_score(self, lhs, rhs, method = "gain"):
        y = self.y[self.idxs] # subset
        if method == "gain":
            lhs_ent = self.entropy(y[lhs])
            rhs_ent = self.entropy(y[rhs])
            value = lhs_ent + rhs_ent
        else:
            lhs_std = y[lhs].std()
            rhs_std = y[rhs].std()
            value = lhs_std * lhs.sum() + rhs_std * rhs.sum()
        return value
                
    @property
    def split_col(self): return self.x[self.idxs,self.var_idx]
                
    @property
    def is_leaf(self): return self.score == float('inf')                

    def predict(self, x):
        return np.array([self.predict_row(xi) for xi in x])

    def predict_row(self, xi):
        if self.is_leaf: return self.val
        node = self.lhs if xi[self.var_idx] <= self.split else self.rhs
        return node.predict_row(xi)

In [770]:
dt = decisionTree(min_leaf = 100)

In [771]:
X, y = data[["x1", "x2"]].values, data["y"].values

In [ ]:
dt.fit(X, y)

In [767]:
preds = dt.predict(X)

In [768]:
preds

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [769]:
np.sum(preds == y)/len(y)

1.0